# Deep Q-Networks (DQNs)

Overview

- The code implements a Deep Q-Network (DQN) agent that uses the Q-learning algorithm for training. 
- The DQN agent learns to play the CartPole-v0 game by interacting with the environment, storing experiences in memory, and updating a neural network model. 
- It uses experience replay to break the correlation between consecutive samples and improve training stability. The training loop runs for a specified number of episodes, and in each episode, the agent selects actions based on an epsilon-greedy strategy, observes rewards and next states, and updates the neural network model using the Q-learning update rule. Overall, the code trains a DQN agent to play the CartPole-v0 game by combining reinforcement learning principles with deep neural networks.

In [ ]:
# Install necessary packages for virtual display
!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install gym

In [ ]:

!pip install tensorflow


In [ ]:
import gym
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque


In [ ]:
# Class representing a Deep Q-Network agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)  # Experience replay memory
        self.gamma = 0.95  # Discount factor for future rewards
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01  # Minimum exploration rate
        self.epsilon_decay = 0.995  # Decay rate for exploration rate
        self.learning_rate = 0.001  # Learning rate for the neural network
        self.model = self.build_model()  # Neural network model for Q-learning

    def build_model(self):
        # Build a neural network model using Keras
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))  # Input layer
        model.add(Dense(24, activation='relu'))  # Hidden layer
        model.add(Dense(self.action_size, activation='linear'))  # Output layer
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))  # Compile the model
        return model

    def remember(self, state, action, reward, next_state, done):
        # Store the experience in the replay memory
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # Choose an action based on epsilon-greedy strategy
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        # Experience replay - update the Q-network using stored experiences
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [ ]:
env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)


In [ ]:
# Set the number of episodes and batch size
episodes = 100
batch_size = 32
total_rewards = []  # List to store the total rewards for each episode

# Run the training loop for the specified number of episodes
for episode in range(1, episodes + 1):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    done = False
    time = 0
    episode_reward = 0
    
    # Execute the episode until a terminal state is reached
    while not done:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        time += 1
        episode_reward += reward

        if done:
            print(f"Episode: {episode}/{episodes}, Score: {time}, Epsilon: {agent.epsilon:.2}")
            break
    
    total_rewards.append(episode_reward)  # Store the total reward for the episode
    
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)  # Perform the experience replay step


In [ ]:
average_score = np.mean(total_rewards)
print(f"Average Score over {episodes} Evaluation Episodes: {average_score}")

Average Score over 100 Evaluation Episodes: 15.68


#### Conclusion

Based on the evaluation of the DQN agent over 100 episodes, the average score achieved was 15.68. This indicates that the agent was able to achieve a relatively low score on average during the evaluation episodes.

The score represents the agent's ability to balance the CartPole for a longer duration during each episode. With an average score of 15.68, it suggests that the agent needs further improvement to achieve better performance and stability in playing the CartPole-v0 game.